## Xception MODEL

In [1]:
# import packages for Xception Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import load_model

import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
train_image_directory = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train"
validation_image_directory = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation"

In [3]:
filename_train = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-train.csv"
df_train = pd.read_csv(filename_train, header = 0)
filename_test = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-test.csv"
df_test = pd.read_csv(filename_test, header = 0)
filename_val = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-validation.csv"
df_val = pd.read_csv(filename_val, header = 0)

In [4]:
df_train.head(10)

,uniqueID,classLabel,classID,source,imageFile
0,2,occluded-specimens,8,L,a1a8b48e8cb142b3.jpg
1,3,microscope-slides,6,L,79599db2ac9092b6.jpg
2,4,illustrations-color,2,BHL,c449696f2f0d0d92.jpg
3,5,illustrations-color,2,P,80a8f4a393b4e08c.jpg
4,6,animal-specimens,0,AK,041a1c6e73313638.jpg
5,8,occluded-specimens,8,L,ccf1b1ccacb8f8b3.jpg
6,11,live-plants,4,US,246c8e9612111a24.jpg
7,12,biocultural-specimens,1,C,b0b337313164a0f0.jpg
8,13,microscope-slides,6,L,b3931bc4cc2b2925.jpg
9,15,illustrations-gray,3,BR,ebf7674e4c2c0e6e.jpg


In [5]:
print(df_train.columns)
print(df_val.columns)
print(df_test.columns)

Index(['uniqueID', 'classLabel', 'classID', 'source', 'imageFile'], dtype='object')
Index(['uniqueID', 'classLabel', 'classID', 'source', 'imageFile'], dtype='object')
Index(['uniqueID', 'imageFile'], dtype='object')


In [9]:
# train an Xception model
# create the base pre-trained model

df_train["classID"] = df_train["classID"].astype(str)
df_val["classID"] = df_val["classID"].astype(str)

base_model = Xception(weights='imagenet', include_top=False)

In [10]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(1, activation='sigmoid')(x)

In [11]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [12]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False

In [13]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [16]:
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    directory=train_image_directory,
    x_col='imageFile',
    y_col='classID',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
    df_val,
    directory=validation_image_directory,
    x_col='imageFile',
    y_col='classID',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

Found 81945 validated image filenames belonging to 10 classes.
Found 10244 validated image filenames belonging to 10 classes.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="imageFile". These filename(s) will be ignored.
  warnings.warn(


In [17]:
# train the model on the new data for a few epochs
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)


/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_37014/3634458605.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
100/100 [==============================] - 175s 2s/step - loss: 0.3512 - accuracy: 0.8943 - val_loss: 0.3293 - val_accuracy: 0.9000
Epoch 2/10
100/100 [==============================] - 171s 2s/step - loss: 0.3287 - accuracy: 0.9000 - val_loss: 0.3287 - val_accuracy: 0.9000
Epoch 3/10
100/100 [==============================] - 173s 2s/step - loss: 0.3276 - accuracy: 0.9000 - val_loss: 0.3280 - val_accuracy: 0.9000
Epoch 4/10
100/100 [==============================] - 174s 2s/step - loss: 0.3273 - accuracy: 0.9000 - val_loss: 0.3279 - val_accuracy: 0.9000
Epoch 5/10
100/100 [==============================] - 173s 2s/step - loss: 0.3271 - accuracy: 0.9000 - val_loss: 0.3272 - val_accuracy: 0.9000
Epoch 6/10
100/100 [==============================] - 177s 2s/step - loss: 0.3269 - accuracy: 0.9000 - val_loss: 0.3271 - val_accuracy: 0.9000
Epoch 7/10
100/100 [==============================] - 181s 2s/step - loss: 0.3269 - accuracy: 0.9000 - val_loss: 0.3282 - val_accuracy: 0.9000

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 Xception blocks, i.e. we will freeze
# the first 115 layers and unfreeze the rest:
for layer in model.layers[:115]:
    layer.trainable = False
for layer in model.layers[115:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use Adam with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

# save the model
model.save('xception_model.h5')